# Book subscription  service research

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px; ">
<b>Project goal:</b>
    
Analysis of the subscription book reading service database
</div>

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px; ">
    
<b>Data description:</b>
    
`books` — contains information about books:

- `book_id` - book identifier;
- `author_id` - author identifier;
- `title` — book title;
- `num_pages` — number of pages; 
- `publication_date` - book publication date;
- `publisher_id` - publisher identifier

`authors` — contains data about authors:

- `author_id` - author identifier;
- `author` — author's name
    
`publishers` — contains information about publishers:

- `publisher_id` - publisher identifier;
- `publisher` - name of the publishing house;

`ratings` — contains data on user ratings of books:

- `rating_id` - rating identifier;
- `book_id` - book identifier;
- `username` - name of the user who left the rating;
- `rating` - book rating

`reviews` — contains data on user reviews of books:

- `review_id` - review identifier;
- `book_id` - book identifier;
- `username` - name of the user who wrote the review;
- `text` — review text
    
<b>Required steps:</b>

-  determine how many books were published after January 1, 2000; 
-  for each book, determine the number of reviews and the average rating; 
-  identify the publishing house that has published the largest number of books thicker than 50 pages
-  determine the author with the highest average book rating, among books with 50 or more ratings; 
-  determine the average number of reviews from users who gave more than 50 ratings

</div>

## Table exploration

Create a connector to the database:

In [2]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': '', 
 'pwd': '', 
 'host': '',
 'port': '', 
 'db': ''} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
def sql_result(query):
    return pd.io.sql.read_sql(query, con = engine)

Let's look at the contents of all tables:

In [4]:
#books
query = '''

select * from books 
limit 5

'''
sql_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
#authors
query = '''

select * from authors
limit 5

'''
sql_result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
#publishers
query = '''

select * from publishers
limit 5

'''
sql_result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
#ratings
query = '''

select * from ratings
limit 5

'''
sql_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
#reviews
query = '''

select * from reviews
limit 5

'''
sql_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Books published after January 1, 2000

In [9]:
query = '''

select count(*) as count_books
from books
where publication_date >'2000-01-01'

'''
sql_result(query)

,count_books
0,819


After January 1, 2000, 819 books were published.

### Number of reviews and average rating for each book

In [10]:
query = '''

select 
b.title,
count(distinct re.review_id) as count_reviews,
avg(ra.rating) as avg_raiting
from books b
inner join ratings ra on b.book_id = ra.book_id
inner join reviews re on b.book_id = re.book_id
group by b.title
--order by avg_raiting desc
order by count_reviews desc

limit 5

'''
sql_result(query)

,title,count_reviews,avg_raiting
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Eat Pray Love,6,3.395833
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
4,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


The most reviews were written for the books “Memoirs of a Geisha”, “Twilight”, “Eat Pray Love”.

### Publisher that has published the largest number of books thicker than 50 pages

In [11]:
query = '''

select 
p.publisher,
count(b.book_id) as count_books
from books b
inner join publishers p on b.publisher_id = p.publisher_id
where  b.num_pages > 50
group by p.publisher
order by count_books desc

limit 1

'''
sql_result(query)

,publisher,count_books
0,Penguin Books,42


The largest number of books was published by Penguin Books (excluding brochures).

### The author with the highest average book rating among books with 50 or more ratings

In [14]:
query = '''

with t as (

select 
a.author,
b.book_id, 
avg(r.rating)
from authors a
inner join books b on a.author_id = b.author_id
inner join ratings r on b.book_id = r.book_id
group by a.author, b.book_id
having count(r.rating) > 50

)

select 
t.author as author,                         
avg(t.avg) as avg_raiting
from t
group by t.author
order by 2 desc



'''
sql_result(query)

,author,avg_raiting
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Authors with the highest average book ratings, among books with 50 or more ratings - J.K. Rowling/Mary GrandPré

### Average number of reviews from users who gave more than 50 ratings

In [13]:
query = '''

with t as (

select 
re.username, 
count(re.review_id) as review_count_avg
from reviews re, (
    select username
    from ratings
    group by username
    having count(rating) > 50) ra
where re.username = ra.username
group by re.username

)
                       
select avg(review_count_avg) from t

'''
sql_result(query)

,avg
0,24.333333


The average number of reviews from users who gave more than 50 ratings is 24

## Conclusions

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px; ">

<li>after January 1, 2000, 819 books were published</li>
<li>most reviews were written for the books "Memoirs of a Geisha", "Twilight", "Eat Pray Love"</li>
<li>Penguin Books has published the most books (excluding brochures)</li>
<li> authors with the highest average book rating, among books with 50 or more ratings - J.K. Rowling/Mary GrandPré</li>
<li>average number of reviews from users who gave more than 50 ratings - 24</li>
</div>